In [3]:
import json, requests
import pandas as pd
import numpy as np
import io
import warnings
warnings.filterwarnings('ignore')

from joblib import dump, load

In [4]:
import matplotlib as mlab
import matplotlib.pyplot as plt
import seaborn as sns 
import time
import math
# import statsmodels.api as sm    
# import scipy.stats as stats 
# from pylab import rcParams

import sys
from functools import reduce
import itertools
import random

import datetime
from datetime import date, timedelta
from pandas.tseries.offsets import Day, MonthBegin, MonthEnd
# Libraries for stock data
import yfinance as yf

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
# from tensorflow.keras.layers import Conv1D,BatchNormalization, Dropout, Flatten, Input, Dense
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# from tensorflow.keras.models import Sequential

2023-01-25 22:31:47.622625: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 22:31:52.865301: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-25 22:31:52.865774: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-25 22:31:52.865784: W 

In [5]:
import random
from random import sample
from transform_forecaster import stock_data_pull, fill_blanks, \
     change_shape, dataset, window_gen; 

Procedure for testing:
1. test model response with randomly generated data.
2. test model with data sourced from y finance (just a hand full of symbols)
3. include pre-process steps similar to those in direction model
4. Test model within .py file

In [50]:
#recon_model = keras.models.load_model('/home/cdsw/saved_models/model_01_17_23')
recon_model = keras.models.load_model('saved_models/model_01_17_23')

**Random input data**

In [51]:
randomlist = []
for i in range(0,60):
  n = random.randint(95,130)
  randomlist.append(n)
# converting list to array
arr = np.array(randomlist)

test = arr.reshape(1,arr.shape[0],1)

pred = recon_model.predict(test)

1/1 [==============================] - 1s 757ms/step


In [52]:
pred

array([[100.5154  , 101.50249 , 101.77298 , 102.7384  , 102.68505 ,
        103.76695 , 103.121346, 102.95126 , 102.59587 , 103.2661  ,
        103.47299 , 102.86976 ]], dtype=float32)

In [9]:
#!ls /home/cdsw/saved_models

**Load yfinance data**

In [10]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
tckr = table[0]
# convert to list
basket = tckr.Symbol.unique().tolist()

In [11]:
smp_basket = sample(basket,10)

In [12]:
# Pull stock data for given ticker symvols
stock_total = stock_data_pull(smp_basket,False);

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Name'], dtype='object')


In [13]:
stock_total.Datetime.drop_duplicates().sort_values()

0      2022-12-27 09:30:00-05:00
1      2022-12-27 09:35:00-05:00
2      2022-12-27 09:40:00-05:00
3      2022-12-27 09:45:00-05:00
4      2022-12-27 09:50:00-05:00
                  ...           
1556   2023-01-25 15:40:00-05:00
1557   2023-01-25 15:45:00-05:00
1558   2023-01-25 15:50:00-05:00
1559   2023-01-25 15:55:00-05:00
1560   2023-01-25 16:00:00-05:00
Name: Datetime, Length: 1561, dtype: datetime64[ns, America/New_York]

In [14]:
window = 60
date_range = stock_total.Datetime.drop_duplicates()
# added line below to clip off the lasest data
date_range = date_range.sort_values().tail(window)
date_range = pd.DataFrame(date_range)
stock_ready = fill_blanks(stock_total,'Name',date_range,'Datetime');

In [15]:
# date_range.sort_values('Datetime').tail(10)
# look at resultant dataframe
stock_ready.head()

stock_ready.dtypes

Datetime    datetime64[ns, America/New_York]
Close                                float64
Volume                                 int64
Name                                  object
dtype: object

In [16]:
# check that all stocks have equal sizes
sizes = stock_ready.groupby('Name').size().unique()
sizes

array([60])

#### Generate predictions for actual stocks

In [17]:
stock_ready.Name.unique().shape

(10,)

In [18]:
symbol_list = []
for i, tckr in enumerate(stock_ready.Name.unique()):
    symbol_list.append(tckr)
    temp_input = stock_ready[stock_ready.Name == tckr].copy().Close.values
    if i == 0:
        input_array = temp_input
    else:
        input_array = np.append(input_array,temp_input)

In [19]:
input_array = input_array.reshape(i+1,window,1)

In [20]:
pred = recon_model.predict(input_array)

1/1 [==============================] - 1s 843ms/step


In [21]:
pred.shape

(10, 12)

In [22]:
i+1

10

In [23]:
#pred.tolist()
pred_time_list = [stock_ready.Datetime.max()]*(i+1)

In [24]:
tst_output = {'symbol':symbol_list,'pred_time':pred_time_list,'prediction':pred.tolist()}

Test .py file with forecating model

In [25]:
from fct_model_serve import forecast_price


Not running in a model replica, so using a local development
version of the model metrics service. Please use the following
CRN's to consume metrics:
   model_crn: "crn:cdp:ml:::workspace:dev/model" (cdsw.dev_model_crn)
   model_build_crn: "crn:cdp:ml:::workspace:dev/model-build" (cdsw.dev_model_build_crn)
   model_deployment_crn: "crn:cdp:ml:::workspace:dev/model-deployment" (cdsw.dev_model_deployment_crn)



#### Test deployed Model

In [28]:
# use this as input data
cols = ['symbol','symbol_time','close', 'volume']
stock_total.columns = ['symbol_time','close','volume','symbol']
stock_total.head()

,symbol_time,close,volume,symbol
0,2022-12-27 09:30:00-05:00,35.080002,1561353,CMCSA
1,2022-12-27 09:35:00-05:00,34.935001,173377,CMCSA
2,2022-12-27 09:40:00-05:00,34.900002,94826,CMCSA
3,2022-12-27 09:45:00-05:00,34.820000,117530,CMCSA
4,2022-12-27 09:50:00-05:00,34.775002,105744,CMCSA


In [29]:
model_url = 'https://modelservice.ml-5fea65b7-abd.loki-cdp.a465-9q4k.cloudera.site/model'
accessKey = 'mjvsvobdvtwsmqu43j8cp6byc8w4ki5r'


In [30]:
stock_total.dtypes

symbol_time    datetime64[ns, America/New_York]
close                                   float64
volume                                    int64
symbol                                   object
dtype: object

In [31]:
cols = stock_total.columns

In [32]:
# load test data
df = pd.read_csv('test_model_input.csv')

In [33]:
df.groupby('Datetime').size()
#df.head()

Datetime
2023-01-09 13:35:00    10
dtype: int64

In [34]:
stock_total.symbol_time = stock_total.symbol_time.astype("str") 

In [35]:
stock_total.dtypes

symbol_time     object
close          float64
volume           int64
symbol          object
dtype: object

In [36]:
# build input for api into dictionary w/ values as a list
input_data_mult = dict()
for col in cols:
    input_data_mult[col] = stock_total[col].to_list()    
    
# build embedded dictionary step 1    
request_dict = {"request":input_data_mult}

# access key will be end point specific
BackDict = {"accessKey":accessKey}
BackDict.update(request_dict)
request_dict=BackDict

In [37]:
type(request_dict)

dict

In [44]:
# request_dict

In [53]:
r = requests.post(model_url, data=json.dumps(request_dict),headers={'Content-Type': 'application/json'})

In [54]:
r.json()

{'success': False,
 'ReplicaID': 'forecasting-model-2-5-32-55f44c759c-fh48w',
 'Size': 139,
 'StatusCode': 400}

In [ ]:
# def mkt_movement(args):
#     filtArgs = {key:[args[key]] for key in features}
#     data = pd.DataFrame.from_dict(filtArgs)
    
#     pipe_rf = load("saved_models/pipe_rf.joblib")

#     prediction = pipe_rf.predict(data)
#     print(prediction)
        
#     return prediction

**TRy with single symbol**

In [34]:
stock_total.columns

Index(['symbol_time', 'close', 'volume', 'symbol'], dtype='object')

In [36]:
#stock_total.symbol.unique()
test_df = stock_total[stock_total['symbol'] == 'SPGI'].copy()

In [42]:
test_df.sort_values('symbol_time').tail()

,symbol_time,close,volume,symbol
1555,2023-01-25 15:35:00-05:00,367.160004,19423,SPGI
1556,2023-01-25 15:40:00-05:00,366.589996,19952,SPGI
1557,2023-01-25 15:45:00-05:00,366.515015,33963,SPGI
1558,2023-01-25 15:50:00-05:00,366.170013,44204,SPGI
1559,2023-01-25 15:55:00-05:00,366.149994,17226,SPGI


In [37]:
# build input for api into dictionary w/ values as a list
input_data_mult = dict()
for col in cols:
    input_data_mult[col] = test_df[col].to_list()   

In [38]:
input_data_mult

{'symbol_time': ['2022-12-27 09:30:00-05:00',
  '2022-12-27 09:35:00-05:00',
  '2022-12-27 09:40:00-05:00',
  '2022-12-27 09:45:00-05:00',
  '2022-12-27 09:50:00-05:00',
  '2022-12-27 09:55:00-05:00',
  '2022-12-27 10:00:00-05:00',
  '2022-12-27 10:05:00-05:00',
  '2022-12-27 10:10:00-05:00',
  '2022-12-27 10:15:00-05:00',
  '2022-12-27 10:20:00-05:00',
  '2022-12-27 10:25:00-05:00',
  '2022-12-27 10:30:00-05:00',
  '2022-12-27 10:35:00-05:00',
  '2022-12-27 10:40:00-05:00',
  '2022-12-27 10:45:00-05:00',
  '2022-12-27 10:50:00-05:00',
  '2022-12-27 10:55:00-05:00',
  '2022-12-27 11:00:00-05:00',
  '2022-12-27 11:05:00-05:00',
  '2022-12-27 11:10:00-05:00',
  '2022-12-27 11:15:00-05:00',
  '2022-12-27 11:20:00-05:00',
  '2022-12-27 11:25:00-05:00',
  '2022-12-27 11:30:00-05:00',
  '2022-12-27 11:35:00-05:00',
  '2022-12-27 11:40:00-05:00',
  '2022-12-27 11:45:00-05:00',
  '2022-12-27 11:50:00-05:00',
  '2022-12-27 11:55:00-05:00',
  '2022-12-27 12:00:00-05:00',
  '2022-12-27 12:05:00-0

In [ ]:

# build embedded dictionary step 1    
request_dict = {"request":input_data_mult}

# access key will be end point specific
BackDict = {"accessKey":accessKey}
BackDict.update(request_dict)
request_dict=BackDict